In [1]:
import dmd_with_power_control_single_task as DMD
import imageio
import numpy as np
import time
from glob import glob
import os
import importlib
importlib.reload(DMD)
import napari as nap

In [2]:
XP_FOLDER = r'D:\Data\power_titration_george' # 
TARGET_FOLDER = XP_FOLDER + r'\8_1__mVenus_20241120_165306' # D:\Data\time_titration_george\1_1__mVenus_20241117_150924
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

mirrors = DMD.dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.
mirrors.idle()

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


c:\Users\Light Sheet User\Documents\GitHub\DMD\dmd_with_power_control_single_task.py:43: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\\\Users\\\\Light Sheet User\\\\Desktop\\\\DMD_parameters.pickled'>
  self.parameters = pickle.load(open(general_parameters_folder, 'rb'))


SELECT ROIs

In [3]:
mirrors.select_ROIs()

IMPORT the ROIs for USAGE

In [4]:
mirrors.import_N_rois(exp='8_1')

Found 2 ROI(s)


In [5]:
masks_path = r'D:\Data\time_titration_george\excitation_patterns\test_dmd'
img1 = imageio.imread(masks_path + '\\mask_1.png') # odor

mirrors.project(img1, 0.1)

DMD ON,  input voltage 0.100000 



C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_25012\670401117.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odor


In [6]:
mirrors.idle()

POWER TITRATION

In [5]:
masks_path = r'D:\Data\power_titration_george\excitation_patterns\8_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\8_1__DAN_20241120_165817' # D:\Data\g12\
protocol_name = r"/8_1"
images_folder = mirrors.xp_folder + current_acquisition

img1 = imageio.imread(masks_path + '\\mask_1.png') # odor
img2 = imageio.imread(masks_path + '\\mask_2.png') # pain

start = time.time()

powers = [0.100, 0.125, 0.150, 0.175, 0.200] # powers without filter (0.1 w/o filter == 2.0 w/ filter)
steps = []

odor_on_time = 15
pun_on_time = 10
off_time = 45

steps.append(time.time())
for i in range(len(powers)):
    mirrors.project(img1, powers[i]); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img1, powers[i]); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img1, powers[i]); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

steps.append(time.time())

for i in range(len(powers)):
    mirrors.project(img2, powers[i]); time.sleep(pun_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, powers[i]); time.sleep(pun_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, powers[i]); time.sleep(pun_on_time); mirrors.idle(); time.sleep(off_time)

steps.append(time.time())

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

timesteps=[]
for t in steps:
    for num, volume in enumerate(all_volumes):
        current = os.path.getctime(volume)
        if current - t > 0:
            timesteps.append(num)
            break

np.savez(images_folder + protocol_name,\
         TM=timesteps, images=["1", "dark", "1", "dark", "1", "dark",
                        "1", "dark", "1", "dark", "1", "dark",
                        "1", "dark", "1", "dark", "1", "dark",
                        "1", "dark", "1", "dark", "1", "dark",
                        "1", "dark", "1", "dark", "1", "dark",
                        "pun", "dark", "pun", "dark", "pun", "dark",
                        "pun", "dark", "pun", "dark", "pun", "dark",
                        "pun", "dark", "pun", "dark", "pun", "dark",
                        "pun", "dark", "pun", "dark", "pun", "dark",
                        "pun", "dark", "pun", "dark", "pun", "dark",
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, odor_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time, pun_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time, pun_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time, pun_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time, pun_on_time, off_time, 
                           pun_on_time, off_time, pun_on_time, off_time, pun_on_time, off_time, 
                           ]
                ) # 28.75 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_27924\3025996532.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odor
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_27924\3025996532.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # pain


DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.200000 

DMD ON,  input voltage 0.200000 

DMD ON,  input voltage 0.200000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.125000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.150000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.175000 

DMD ON,  input voltage 0.200000 

DMD ON,  input voltage 0.200000 

DMD ON,  input

Time Titration

In [5]:
masks_path = r'D:\Data\time_titration_george\excitation_patterns\8_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\8_1__DAN_20241120_145626' # D:\Data\g12\
protocol_name = r"/8_1"
images_folder = mirrors.xp_folder + current_acquisition

img1 = imageio.imread(masks_path + '\\mask_1.png') # odor 1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odor 2

start = time.time()

power = 0.1 # powers without filter (0.1 w/o filter == 2.0 w/ filter)
steps = []

odor_on_time = 15
isi_times = [45, 60, 75]
buffer_off_time = 90

steps.append(time.time())
for i in range(len(isi_times)):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(isi_times[i])
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(isi_times[i])
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(isi_times[i])
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(isi_times[i])
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(isi_times[i])
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(buffer_off_time)
    steps.append(time.time())


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

timesteps=[]
for t in steps:
    for num, volume in enumerate(all_volumes):
        current = os.path.getctime(volume)
        if current - t > 0:
            timesteps.append(num)
            break

np.savez(images_folder + protocol_name,\
         TM=timesteps, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark", 
                                "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark", 
                                "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark", 
                         ], \
                durations=[odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, buffer_off_time,
                           odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, buffer_off_time,
                           odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, buffer_off_time,
                           ]
                ) # 24 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_24528\3576001331.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odor 1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_24528\3576001331.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odor 2


DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 

DMD ON,  input voltage 0.100000 



Time Calculation

In [9]:
odor_on_time = 15
isi_times = [45, 60, 75]
buffer_off_time = 90

durations=[odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, isi_times[0], odor_on_time, buffer_off_time,
                           odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, isi_times[1], odor_on_time, buffer_off_time,
                           odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, isi_times[2], odor_on_time, buffer_off_time,
                         
                           ]
t = 0
for i in durations:
    t+= i


print(t/60)

24.0
